In [339]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [340]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [341]:
spark

# Leyendo lista de Colegios y Estudiantes

In [342]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [343]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [344]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [345]:
print("Archivos de estudiantes: ", estuDic['20141'],estuDic['20142'])
print("Archivos de colegios: ",coleDic['20141'],coleDic['20142'])

Archivos de estudiantes:  SB11_20141.txt SB11_20142.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-20141.txt SB11-CLASIFI-PLANTELES-20142.txt


In [346]:
df1 = spark.read.load(base2 + '/' + estuDic['20141'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [347]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_VECESPRESENTOEXAMEN',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENETELEVISOR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_RECIBESALARIO',
 'ESTU_ANTECEDENTES',
 'ESTU_ANOSPREESCOLAR',
 'ESTU_ANOMATRICULAPRIMERO',
 'ESTU_ANOTERMINOQUINTO',
 'ESTU_ANOMATRICULASEXTO',
 'ES

In [348]:
columnasPuntos20141 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC']

In [349]:
df1 = df1.select(columnasPuntos20141)

In [350]:
# SE leen los dos archivos y se crean dos dataframes
df2 = spark.read.load(base2 + '/' + estuDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [351]:
columnasPuntos20142 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_LECTURA_CRITICA',
 'PUNT_MATEMATICAS',
 'PUNT_C_NATURALES',
 'PUNT_SOCIALES_CIUDADANAS',
 'PUNT_RAZONA_CUANTITATIVO',
 'PUNT_COMP_CIUDADANA',
 'PUNT_INGLES'
]

In [352]:
df2 = df2.select(columnasPuntos20142)

In [353]:
#dft = df1.columns.remove('cole_cod_dane_institucion')
cols = df1.columns
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols.remove('COLE_NOMBRE_ESTABLECIMIENTO')
cols

['PUNT_MATEMATICAS',
 'PUNT_PROFUNDIZA_MATEMATICA',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_INGLES',
 'PUNT_BIOLOGIA',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_INTERDISC_VIOLENCIAYSOC']

In [354]:
# SE unen los dos dataframes
# df = unionAll(df1,df2)
for cp in cols:
    df1 = df1.withColumn(cp, regexp_replace(cp, ',', '.'))
    df1 = df1.withColumn(cp, df1[cp].cast("float"))
    df1 = df1.na.fill({cp:0})

In [355]:
df1 = df1.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df1['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [356]:
cols2 = df2.columns
cols2.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols2.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [357]:
for cp in cols2:
    df2 = df2.withColumn(cp, regexp_replace(cp, ',', '.'))
    df2 = df2.withColumn(cp, df2[cp].cast("float"))
    df2 = df2.na.fill({cp:0})
df2 = df2.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df2['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [358]:
print("Numero de filas primero semestre ", df1.count())
print("Numero de filas segundo semestre ", df2.count())

Numero de filas primero semestre  24893
Numero de filas segundo semestre  544401


In [359]:
print(df1.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_PROFUNDIZA_MATEMATICA: float (nullable = false)
 |-- PUNT_CIENCIAS_SOCIALES: float (nullable = false)
 |-- PUNT_PROFUNDIZA_CSOCIALES: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)
 |-- PUNT_BIOLOGIA: float (nullable = false)
 |-- PUNT_PROFUNDIZA_BIOLOGIA: float (nullable = false)
 |-- PUNT_FILOSOFIA: float (nullable = false)
 |-- PUNT_INTERDISC_MEDIOAMBIENTE: float (nullable = false)
 |-- PUNT_FISICA: float (nullable = false)
 |-- PUNT_QUIMICA: float (nullable = false)
 |-- PUNT_LENGUAJE: float (nullable = false)
 |-- PUNT_PROFUNDIZA_LENGUAJE: float (nullable = false)
 |-- PUNT_INTERDISC_VIOLENCIAYSOC: float (nullable = false)

None


In [360]:
print(df2.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_LECTURA_CRITICA: float (nullable = false)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_C_NATURALES: float (nullable = false)
 |-- PUNT_SOCIALES_CIUDADANAS: float (nullable = false)
 |-- PUNT_RAZONA_CUANTITATIVO: float (nullable = false)
 |-- PUNT_COMP_CIUDADANA: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)

None


In [361]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)
dsCountEst2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [362]:
dsCountEst1.show(10,False)
# Numero de estudiantes por instituciòn primer semestre

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|311001105863                 |465  |
|305001024544                 |326  |
|305001007861                 |267  |
|476248000478                 |258  |
|376001028219                 |204  |
|376001000721                 |176  |
|376001003461                 |168  |
|311001106410                 |159  |
|376001007670                 |150  |
|376001026925                 |142  |
+-----------------------------+-----+
only showing top 10 rows



In [363]:
dsCountEst1.count()

798

In [364]:
dsCountEst2.show(10,False)
# Numero de estudiantes por instituciòn segundo semestre

+-----------------------------+-----+
|COLE_COD_DANE_ESTABLECIMIENTO|count|
+-----------------------------+-----+
|311001105391                 |1133 |
|105001000108                 |1001 |
|311001105863                 |957  |
|105001013340                 |794  |
|105615000236                 |671  |
|305001007861                 |663  |
|176001005813                 |648  |
|305001003963                 |608  |
|105154000301                 |572  |
|405001016946                 |551  |
+-----------------------------+-----+
only showing top 10 rows



In [365]:
#Se crea una columna con el total de los puntajes 
df1= df1.withColumn("Suma",df1['PUNT_MATEMATICAS']+df1['PUNT_PROFUNDIZA_MATEMATICA']+df1['PUNT_CIENCIAS_SOCIALES']+
                     df1['PUNT_PROFUNDIZA_CSOCIALES']+df1['PUNT_INGLES']+df1['PUNT_BIOLOGIA']+df1['PUNT_PROFUNDIZA_BIOLOGIA']+
                     df1['PUNT_FILOSOFIA']+df1['PUNT_INTERDISC_MEDIOAMBIENTE']+df1['PUNT_FISICA']+df1['PUNT_QUIMICA']+
                     df1['PUNT_LENGUAJE']+df1['PUNT_PROFUNDIZA_LENGUAJE']+df1['PUNT_INTERDISC_VIOLENCIAYSOC'])

df2= df2.withColumn("Suma",df2['PUNT_LECTURA_CRITICA']+df2['PUNT_MATEMATICAS']+df2['PUNT_C_NATURALES']
                    +df2['PUNT_SOCIALES_CIUDADANAS']+df2['PUNT_RAZONA_CUANTITATIVO']+df2['PUNT_COMP_CIUDADANA']
                    +df2['PUNT_INGLES'])

## Resultados finales por instituciòn 1er semestre

In [366]:
# Còdigo para obtener la media de los resultados de las evaluaciones de todos los estudiantes por instituciòn 
# y posiciòn en orden descendente.
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df1 = df1.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()
df1 = df1.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df1 = df1.orderBy('rank',ascending= True)

In [367]:
df1.select('COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+----------------------------------------------------------------------------------------------------+-----------------+----+
|COLE_NOMBRE_ESTABLECIMIENTO                                                                         |avg(Suma)        |rank|
+----------------------------------------------------------------------------------------------------+-----------------+----+
|COL LOS NOGALES                                                                                     |594.719298245614 |1   |
|COL LA QUINTA DEL PUENTE                                                                            |586.7666666666667|2   |
|COL SAN MATEO APOSTOL                                                                               |576.2131147540983|3   |
|COL  MONTESSORI BRITISH SCHOOL                                                                      |570.4285714285714|4   |
|COL SAN CARLOS                                                                                      |569.989130434782

## Resultados finales por instituciòn 2do semestre

In [368]:
df2 = df2.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()
df2 = df2.withColumn("rank", dense_rank().over(Window.orderBy(desc("avg(Suma)"))))
df2 = df2.orderBy('rank',ascending= True)

In [369]:
df2.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','rank').show(10,False)

+-----------------------------+-------------------------------------------+-----------------+----+
|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO                |avg(Suma)        |rank|
+-----------------------------+-------------------------------------------+-----------------+----+
|311001086320                 |COLOMBO BRITANICO - BILINGÜE INTERNACIONAL |541.5833333333334|1   |
|354518001139                 |CAMBRIDGE SCHOOL (GIMN BILING PLAZA SESAMO)|538.5714285714286|2   |
|311001005176                 |LIC NAVARRA                                |534.0888888888888|3   |
|311001089221                 |INST ALBERTO MERANI                        |530.40625        |4   |
|311001076634                 |COL NUEVO COLOMBO AMERICANO                |527.8615384615384|5   |
|311001065489                 |LIC CAMPO DAVID                            |526.3529411764706|6   |
|311848000308                 |COL ANGLO AMERICANO                        |524.3674698795181|7   |
|425214000

## Colegios primer semestre

In [370]:
# SE crea Dataframe de colegios primer semestre
dfc = spark.read.load(base + '/' + coleDic['20141'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='\t')
dfc = dfc.withColumn('COLE_COD_DANE', dfc['COLE_COD_DANE'].cast("string"))

In [371]:
print(dfc.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- COLE_ESTUDIANTES_PRESENTES: integer (nullable = true)

None


In [372]:
colsColegio = dfc.columns

In [373]:
dfc.select('COLE_COD_DANE','COLE_INST_NOMBRE','COLE_ESTUDIANTES_PRESENTES').show(10)

+-------------+--------------------+--------------------------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|COLE_ESTUDIANTES_PRESENTES|
+-------------+--------------------+--------------------------+
|        86769|COLEGIO SAN LUIS ...|                         7|
|        36889|COLEGIO LOS ANGEL...|                        52|
|       150581|  COLEGIO SAN RAFAEL|                        19|
|       136036|CENT EDUC DIST CA...|                        37|
|       136952|CENT EDUC DIST CA...|                        20|
|         4986|IE TECNICA PIO AL...|                        30|
|         5785|INSTITUCION EDUCA...|                        24|
|        41459|COLEGIO COMERCIAL...|                        11|
|       158725|           SIGLO XXI|                        13|
|       106120|EL NUEVO COL THE ...|                        47|
+-------------+--------------------+--------------------------+
only showing top 10 rows



In [374]:
print("El numero de colegios es: ", dfc.count())

El numero de colegios es:  365


## Colegios segundo semestre

In [375]:
# SE crea Dataframe de colegios primer semestre
dfc2 = spark.read.load(base + '/' + coleDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))

In [376]:
print(dfc2.printSchema())

root
 |-- PERIODO: integer (nullable = true)
 |-- COLE_COD_DANE: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_MUNICIPIO: string (nullable = true)
 |-- COLE_COD_DEPTO: integer (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GRADO: integer (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- MATRICULADOS_ULTIMOS_3: integer (nullable = true)
 |-- EVALUADOS_ULTIMOS_3: integer (nullable = true)
 |-- INDICE_MATEMATICAS: string (nullable = true)
 |-- INDICE_C_NATURALES: string (nullable = true)
 |-- INDICE_SOCIALES_CIUDADANAS: string (nullable = true)
 |-- INDICE_LECTURA_CRITICA: string (nullable = true)
 |-- INDICE_INGLES: string (nullable = true)
 |-- INDICE_TOTAL: string (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)

None


In [377]:
dfc2.select('COLE_COD_DANE','COLE_INST_NOMBRE','INDICE_TOTAL').show(10)

+-------------+--------------------+------------+
|COLE_COD_DANE|    COLE_INST_NOMBRE|INDICE_TOTAL|
+-------------+--------------------+------------+
| 205790000235| I. E. LA INMACULADA|      0,6112|
| 154128000019|INST EDUCATIVA CO...|      0,6924|
| 108433000051|INSTITUCION EDUCA...|      0,6012|
| 105147000568|I. E. JOSE MARIA ...|      0,6692|
| 105147000401|      I. E. COLOMBIA|      0,6408|
| 219110000636|INSTITUCION EDUCA...|      0,5284|
| 419110000970|          I.E. TIMBA|      0,6417|
| 111001104264|COLEGIO GERARDO M...|      0,6945|
| 205138000142|I. E. R. BUENOS A...|      0,6721|
| 276364001458|    I.E. SAN ANTONIO|       0,607|
+-------------+--------------------+------------+
only showing top 10 rows



In [378]:
print("El numero de colegios segundo semestre: ", dfc2.count())

El numero de colegios segundo semestre:  10927


## Anàlisis comparativo entre estudiantes y colegios primer semestre

In [379]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_COD_DANE)

In [380]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 0


In [381]:
dfjoinPorNom = df1.join(dfc, df1.COLE_NOMBRE_ESTABLECIMIENTO == dfc.COLE_INST_NOMBRE)

In [385]:
dfjoinPorNom.printSchema()

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- avg(PUNT_MATEMATICAS): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_MATEMATICA): double (nullable = true)
 |-- avg(PUNT_CIENCIAS_SOCIALES): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_CSOCIALES): double (nullable = true)
 |-- avg(PUNT_INGLES): double (nullable = true)
 |-- avg(PUNT_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_FILOSOFIA): double (nullable = true)
 |-- avg(PUNT_INTERDISC_MEDIOAMBIENTE): double (nullable = true)
 |-- avg(PUNT_FISICA): double (nullable = true)
 |-- avg(PUNT_QUIMICA): double (nullable = true)
 |-- avg(PUNT_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_INTERDISC_VIOLENCIAYSOC): double (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- rank: integer (nullable = true)
 |-- PERIODO

In [382]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios:", dfjoinPorNom.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 1er semestre y colegios: 169


In [386]:
# Se cruza la informaciòn por el nombre del colegio - se evidencia que el còdigo DANE no coincide
dfjoinPorNom.select('rank','COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE').show(10)

+----+-----------------------------+---------------------------+-------------+
|rank|COLE_COD_DANE_ESTABLECIMIENTO|COLE_NOMBRE_ESTABLECIMIENTO|COLE_COD_DANE|
+----+-----------------------------+---------------------------+-------------+
|   2|                 368276000826|       COL LA QUINTA DEL...|        55129|
|   3|                 311769004292|       COL SAN MATEO APO...|        62497|
|   5|                 311848002424|       COL SAN CARLOS   ...|        19471|
|   6|                 311848000278|       GIMN VERMONT     ...|        25130|
|  10|                 366001003814|       COL FUNDACION LIC...|        77776|
|  11|                 376001016334|       COLEGIO INGLES DE...|        58719|
|  13|                 376001026968|            COLEGIO FREINET|       105742|
|  15|                 376001011464|       COLEGIO GIMNASIO ...|        55806|
|  18|                 376001002376|       COLEGIO HISPANOAM...|        17004|
|  20|                 305001015503|       COLEGIO L

## Anàlisis comparativo entre estudiantes y colegios segundo semestre

In [120]:
# SE crea Dataframe de colegios segundo semestre
dfc2 = spark.read.load(base + '/' + coleDic['20142'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc2 = dfc2.withColumn('COLE_COD_DANE', dfc2['COLE_COD_DANE'].cast("string"))

In [124]:
dfjoin2 = df2.join(dfc2, df2.COLE_COD_DANE_ESTABLECIMIENTO == dfc2.COLE_COD_DANE)

In [125]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios:", dfjoin2.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes 2do semestre y colegios: 10177


In [132]:
dfjoin2.columns

['COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'avg(PUNT_LECTURA_CRITICA)',
 'avg(PUNT_MATEMATICAS)',
 'avg(PUNT_C_NATURALES)',
 'avg(PUNT_SOCIALES_CIUDADANAS)',
 'avg(PUNT_RAZONA_CUANTITATIVO)',
 'avg(PUNT_COMP_CIUDADANA)',
 'avg(PUNT_INGLES)',
 'avg(Suma)',
 'rank',
 'PERIODO',
 'COLE_COD_DANE',
 'COLE_INST_NOMBRE',
 'COLE_CODMPIO_COLEGIO',
 'COLE_MPIO_MUNICIPIO',
 'COLE_COD_DEPTO',
 'COLE_DEPTO_COLEGIO',
 'COLE_NATURALEZA',
 'COLE_GRADO',
 'COLE_CALENDARIO_COLEGIO',
 'COLE_GENEROPOBLACION',
 'MATRICULADOS_ULTIMOS_3',
 'EVALUADOS_ULTIMOS_3',
 'INDICE_MATEMATICAS',
 'INDICE_C_NATURALES',
 'INDICE_SOCIALES_CIUDADANAS',
 'INDICE_LECTURA_CRITICA',
 'INDICE_INGLES',
 'INDICE_TOTAL',
 'COLE_CATEGORIA']

In [137]:
dfjoin2 = dfjoin2.withColumn("rankIndiceTotal", dense_rank().over(Window.orderBy(desc("INDICE_TOTAL"))))
dfjoin2.select('rank','avg(Suma)','COLE_NOMBRE_ESTABLECIMIENTO','COLE_COD_DANE','INDICE_TOTAL').show(10)

+----+-----------------+---------------------------+-------------+------------+
|rank|        avg(Suma)|COLE_NOMBRE_ESTABLECIMIENTO|COLE_COD_DANE|INDICE_TOTAL|
+----+-----------------+---------------------------+-------------+------------+
|   1|541.5833333333334|       COLOMBO BRITANICO...| 311001086320|      0,9101|
|   2|538.5714285714286|       CAMBRIDGE SCHOOL ...| 354518001139|      0,9104|
|   3|534.0888888888888|                LIC NAVARRA| 311001005176|      0,9011|
|   4|        530.40625|        INST ALBERTO MERANI| 311001089221|       0,913|
|   5|527.8615384615384|       COL NUEVO COLOMBO...| 311001076634|      0,9005|
|   6|526.3529411764706|            LIC CAMPO DAVID| 311001065489|      0,9063|
|   7|524.3674698795181|        COL ANGLO AMERICANO| 311848000308|      0,8967|
|   8|523.8055555555555|                 COL REFOUS| 425214000149|      0,8965|
|   9|519.9060402684564|        COLEGIO CORAZONISTA| 311001090921|      0,9001|
|  10|516.6190476190476|             COL